In [11]:
!pip install neo4j


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd

In [6]:
data=pd.read_csv("C:/Users/thora/Downloads/kg.csv")

C:\Users\thora\AppData\Local\Temp\ipykernel_16972\963437689.py:1: DtypeWarning: Columns (3,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("C:/Users/thora/Downloads/kg.csv")


In [7]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8100498 entries, 0 to 8100497
Data columns (total 12 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   relation          object
 1   display_relation  object
 2   x_index           int64 
 3   x_id              object
 4   x_type            object
 5   x_name            object
 6   x_source          object
 7   y_index           int64 
 8   y_id              object
 9   y_type            object
 10  y_name            object
 11  y_source          object
dtypes: int64(2), object(10)
memory usage: 741.6+ MB


In [8]:
data.head()

,relation,display_relation,x_index,x_id,x_type,x_name,x_source,y_index,y_id,y_type,y_name,y_source
0,protein_protein,ppi,0,9796,gene/protein,PHYHIP,NCBI,8889,56992,gene/protein,KIF15,NCBI
1,protein_protein,ppi,1,7918,gene/protein,GPANK1,NCBI,2798,9240,gene/protein,PNMA1,NCBI
2,protein_protein,ppi,2,8233,gene/protein,ZRSR2,NCBI,5646,23548,gene/protein,TTC33,NCBI
3,protein_protein,ppi,3,4899,gene/protein,NRF1,NCBI,11592,11253,gene/protein,MAN1B1,NCBI
4,protein_protein,ppi,4,5297,gene/protein,PI4KA,NCBI,2122,8601,gene/protein,RGS20,NCBI


In [9]:
print(data['relation'].unique())
print(len(data['relation'].unique()))


['protein_protein' 'drug_protein' 'contraindication' 'indication'
 'off-label use' 'drug_drug' 'phenotype_protein' 'phenotype_phenotype'
 'disease_phenotype_negative' 'disease_phenotype_positive'
 'disease_protein' 'disease_disease' 'drug_effect' 'bioprocess_bioprocess'
 'molfunc_molfunc' 'cellcomp_cellcomp' 'molfunc_protein'
 'cellcomp_protein' 'bioprocess_protein' 'exposure_protein'
 'exposure_disease' 'exposure_exposure' 'exposure_bioprocess'
 'exposure_molfunc' 'exposure_cellcomp' 'pathway_pathway'
 'pathway_protein' 'anatomy_anatomy' 'anatomy_protein_present'
 'anatomy_protein_absent']
30


In [ ]:
NEO4J_URI=
NEO4J_USERNAME=neo4j
NEO4J_PASSWORD=
AURA_INSTANCEID=e20d126f
AURA_INSTANCENAME=Instance02

In [22]:
from neo4j import GraphDatabase

uri = "neo4j+ssc://e20d126f.databases.neo4j.io"
user = "neo4j"
password = "MX-lvMm1vPsqUXKK56xZyJj1i-zyu-fEBqWq-Ex8cdM"

try:
    driver = GraphDatabase.driver(uri, auth=(user, password))
    with driver.session() as session:
        result = session.run("RETURN 'Connection successful!' AS message")
        for record in result:
            print(record["message"])
except Exception as e:
    print(f"Error: {e}")
finally:
    driver.close()

Connection successful!


In [23]:
from neo4j import GraphDatabase
import pandas as pd

# Connect to the Neo4j database
class Neo4jConnection:
    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def close(self):
        self._driver.close()
    
    def query(self, query, parameters=None):
        with self._driver.session() as session:
            return session.run(query, parameters)


# Connect to your Neo4j instance (update with your credentials)
conn = Neo4jConnection(uri="neo4j+ssc://e20d126f.databases.neo4j.io", user="neo4j", password="MX-lvMm1vPsqUXKK56xZyJj1i-zyu-fEBqWq-Ex8cdM")

# Load the CSV file into a Pandas DataFrame
print(conn)
length=len(data.iterrows())
# Create nodes and relationships in Neo4j
for index, row in data.iterrows():
    if length%500==0:
        print(f"\r{index}/{length}", end="")
    # Create nodes for 'x' entities
    conn.query(
        """
        MERGE (x:Entity {name: $x_name})
        SET x.type = $x_type, x.source = $x_source
        """,
        parameters={
            "x_name": row["x_name"],
            "x_type": row["x_type"],
            "x_source": row["x_source"],
        },
    )
    
    # Create nodes for 'y' entities
    conn.query(
        """
        MERGE (y:Entity {name: $y_name})
        SET y.type = $y_type, y.source = $y_source
        """,
        parameters={
            "y_name": row["y_name"],
            "y_type": row["y_type"],
            "y_source": row["y_source"],
        },
    )

    
    # Create relationships between 'x' and 'y'
    conn.query(
        """
        MATCH (x:Entity {name: $x_name}), (y:Entity {name: $y_name})
        MERGE (x)-[r:RELATION {type: $relation}]->(y)
        """,
        parameters={
            "x_name": row["x_name"],
            "y_name": row["y_name"],
            "relation": row["display_relation"],
        },
    )

print("Knowledge graph successfully created!")

# Close the connection
conn.close()

[#F448]  _: <CONNECTION> error: Failed to read from defunct connection IPv4Address(('e20d126f.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.124.169.171', 7687))): OSError('No data')


SessionExpired: Failed to read from defunct connection IPv4Address(('e20d126f.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.124.169.171', 7687)))